<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. Paulina Salcedo Celis  - if708079@iteso.mx </font>

<center><font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center><font color=#42c29b><font size=5> Laboratorio 1 </font> <br> <br>

<center> <font color= #047CFB> <font size = 4> Otoño 2020 </font>  |  <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/'>https://github.com/psc9718/myst_if708079_lab1</a></font>

# Introducción

En este laboratorio comparamos dos estrategias: Inversión Activa e Inversión Pasiva. 

Poder responder la pregunta ¿Qué estrategia de inversión propondrías si tu trabajo fuera administrar 1 Millón de pesos?

Caso: Tienes 1 Millón de pesos, tienes dos opciones para invertirlos. Por un lado, puedes hacer una Inversión Pasiva del capital, comprando el 100% del capital de títulos de un ETF que replica al mercado de renta variable mexicano S&P/BMV IPC que es el NAFTRAC. Pero por otro lado, puedes hacer una Inversión Activa del capital, al crear un portafolio que tenga los mismos activos que el ETF y haciendo el rebalanceo por medio de un algoritmo en específico.

# Objetivos
*Poder entender como es la inversión pasiva. 
*Poder entender como es la inversión activa. 
*Comparar los dos escenarios con el fin de ver cual sería la mejor opción. 

# Datos utilizados
NAFTRAC desde el 2018-2020.
Datos obtenidos de Yahoo Finance. 

# Proceso 



poner lo de main e ir explicando cada paso

# Conclusión
